In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Here we are converting the dataset into two variations:

1. Keeping the Z_Score_Words and removing the Words column(dataset_z)
2. Keeping Words and leaving Z_Score_Words(dataset_w)

In both cases, we will not consider the z-score of helpful votes in this notebook.

In [15]:
dataset = pd.read_csv(r'C:\Users\aksha\PycharmProjects\FinalProject_WorkingCopy\FinalFeatures.csv')

dataset_z = dataset[['Stars','Z_Score_Words', 'Paragraphs','No.break tags','Percentage_Upper_Case','Percentage_Lower_Case','Avg_len_paragraph_per_review','Helpful Votes']]
dataset_z.head()

,Stars,Z_Score_Words,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Helpful Votes
0,3,6.453577,1,0,3,93,3087.000000,837
1,5,1.394079,3,4,3,91,300.000000,374
2,4,3.666459,4,6,4,90,468.500000,263
3,4,8.525083,11,20,3,91,394.272727,200
4,5,1.795826,2,1,6,91,492.000000,247


In [16]:
dataset_w = dataset[['Stars','Words', 'Paragraphs','No.break tags','Percentage_Upper_Case','Percentage_Lower_Case','Avg_len_paragraph_per_review','Helpful Votes']]
dataset_w.head()

,Stars,Words,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Helpful Votes
0,3,565,1,0,3,93,3087.000000,837
1,5,162,3,4,3,91,300.000000,374
2,4,343,4,6,4,90,468.500000,263
3,4,730,11,20,3,91,394.272727,200
4,5,194,2,1,6,91,492.000000,247


Scaling both the datasets below using the MinMaxScaler from sklearn

In [17]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

In [19]:
# for dataset with z-score of words

dataset_new_z = scaler.fit_transform(dataset_z)
dataset_new_z = pd.DataFrame(dataset_new_z,columns=dataset_z.columns)
dataset_new_z.head()

,Stars,Z_Score_Words,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Helpful Votes
0,0.50,0.315262,0.000000,0.000000,0.03,0.93,0.417478,0.167433
1,1.00,0.091377,0.015873,0.026846,0.03,0.91,0.040449,0.074815
2,0.75,0.191931,0.023810,0.040268,0.04,0.90,0.063244,0.052611
3,0.75,0.406928,0.079365,0.134228,0.03,0.91,0.053202,0.040008
4,1.00,0.109154,0.007937,0.006711,0.06,0.91,0.066423,0.049410


In [20]:
# for dataset with just the words

dataset_new_w = scaler.fit_transform(dataset_w)
dataset_new_w = pd.DataFrame(dataset_new_w,columns=dataset_w.columns)
dataset_new_w.head()

,Stars,Words,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Helpful Votes
0,0.50,0.211950,0.000000,0.000000,0.03,0.93,0.417478,0.167433
1,1.00,0.060504,0.015873,0.026846,0.03,0.91,0.040449,0.074815
2,0.75,0.128523,0.023810,0.040268,0.04,0.90,0.063244,0.052611
3,0.75,0.273957,0.079365,0.134228,0.03,0.91,0.053202,0.040008
4,1.00,0.072529,0.007937,0.006711,0.06,0.91,0.066423,0.049410


Now, we will split all the features from the target variable which is "Helpful Votes"

In [22]:
# for dataset with z-score of words

X_z = dataset_new_z.iloc[:,0:-1].values
y_z = dataset_new_z.iloc[:,-1].values

In [23]:
# for dataset with words

X_w = dataset_new_w.iloc[:,0:-1].values
y_w = dataset_new_w.iloc[:,-1].values

Splitting the data into training and testing sets

In [25]:
from sklearn.model_selection import train_test_split

In [27]:
# for dataset with the z-score of words

X_train_z,X_test_z,y_train_z,y_test_z = train_test_split(X_z,y_z, test_size=0.2,random_state=0)

In [28]:
# for dataset with words

X_train_w,X_test_w,y_train_w,y_test_w = train_test_split(X_w,y_w, test_size=0.2,random_state=0)

If we want to use StandardScaler for scaling the data instead of MinMaxScaler, this code can be used

In [29]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.fit_transform(X_test)

In [30]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


Now, we will build the ANN for our regression task.

Since we have already tried the ANN with less number of nodes and the relusts were not impressive, we will use a high number of nodes and look at the results.

In [36]:
regressor = Sequential()

regressor.add(Dense(100, kernel_initializer = 'uniform', activation = 'relu', input_dim = 7))
regressor.add(Dense(100, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dense(1, kernel_initializer = 'uniform'))

The Dense class that we've imported above will help us initilize the neural network with small weights close to 0 but not 0

We did not provide any activation function in the output layer since we are not classifying the data. For classification, we need to provide an activation function for the output layer, whereas we do not need to provide one for regression models

In [37]:
# Compiling the regressor here

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

Now that we have compiled the regressor, we can fit the Artificial Neural Network to out training data

In [40]:
# fitting the ANN on the training dataset X_z( the one with z-score of words)

regressor.fit(X_train_z, y_train_z, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
24362/24362 [==============================] - 4s 154us/step - loss: 8.6169e-05
Epoch 2/100
24362/24362 [==============================] - 4s 145us/step - loss: 8.4383e-05
Epoch 3/100
24362/24362 [==============================] - 4s 145us/step - loss: 8.4896e-05 0s - loss: 8.5322e-0
Epoch 4/100
24362/24362 [==============================] - 4s 145us/step - loss: 8.5163e-05
Epoch 5/100
24362/24362 [==============================] - 4s 146us/step - loss: 8.4739e-05
Epoch 6/100
24362/24362 [==============================] - 4s 150us/step - loss: 8.1766e-05
Epoch 7/100
24362/24362 [==============================] - 4s 145us/step - loss: 7.4757e-05 0s - loss: 4.10
Epoch 8/100
24362/24362 [==============================] - 4s 145us/step - loss: 7.5962e-05 - ETA: 0s - lo
Epoch 9/100
24362/24362 [==============================] - 4s 146us/step - loss: 8.5334e-05 0
Epoch 10/100
24362/24362 [==============================] - 4s 146us/step - loss: 8.4692e-05 2s - loss:  - ETA: 1s - l

In [42]:
y_pred_z = regressor.predict(X_test_z)

In [41]:
# fitting the ANN for the dataset with just the words

regressor.fit(X_train_w, y_train_w, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
24362/24362 [==============================] - 4s 171us/step - loss: 8.6913e-05 0s - loss
Epoch 2/100
24362/24362 [==============================] - 4s 159us/step - loss: 8.5888e-05 
Epoch 3/100
24362/24362 [==============================] - 4s 155us/step - loss: 8.5427e-05
Epoch 4/100
24362/24362 [==============================] - 4s 161us/step - loss: 8.4904e-05 1s
Epoch 5/100
24362/24362 [==============================] - 4s 162us/step - loss: 8.6864e-05 0s - loss: 8.8620e-
Epoch 6/100
24362/24362 [==============================] - 4s 158us/step - loss: 8.6325e-05 0s - loss: 9.0169
Epoch 7/100
24362/24362 [==============================] - 4s 156us/step - loss: 8.5826e-05
Epoch 8/100
24362/24362 [==============================] - 4s 162us/step - loss: 8.5983e-05
Epoch 9/100
24362/24362 [==============================] - 4s 159us/step - loss: 8.5813e-05 0s - loss: 8.7186e-
Epoch 10/100
24362/24362 [==============================] - 4s 155us/step - loss: 8.5250e-05
Epoch 1

In [43]:
y_pred_w = regressor.predict(X_test_w)

Now, we can create a pandas dataframe with the actual values compared with the target variabes that have been predicted using both the datasets.

In [52]:
y_pred_z=y_pred_z.reshape(6091,)
y_pred_w=y_pred_w.reshape(6091,)
print(y_pred_z.shape)
print(y_pred_w.shape)
temp={'Actual Values': y_test_z,'Pred-Dataset with z-scores': y_pred_z,'Pred-Dataset with words only': y_pred_w}
y_compare = pd.DataFrame(temp)

# Calculating the Mean Squared Error and adding the columns with the MSE
y_compare['MSE-> z-score'] = (((y_compare['Actual Values'].values - y_compare['Pred-Dataset with z-scores'].values ) ** 2).mean() ** .5)
y_compare['MSE-> just words'] = (((y_compare['Actual Values'].values - y_compare['Pred-Dataset with words only'].values ) ** 2).mean() ** .5)
y_compare.head(20)


(6091,)
(6091,)


,Actual Values,Pred-Dataset with words only,Pred-Dataset with z-scores,MSE-> z-score,MSE-> just words
0,0.0002,-0.005598,-0.005598,0.006071,0.006071
1,0.0000,-0.005475,-0.005475,0.006071,0.006071
2,0.0000,-0.005396,-0.005396,0.006071,0.006071
3,0.0000,-0.005301,-0.005301,0.006071,0.006071
4,0.0000,-0.005349,-0.005349,0.006071,0.006071
5,0.0000,-0.004298,-0.004298,0.006071,0.006071
6,0.0000,-0.005782,-0.005782,0.006071,0.006071
7,0.0000,-0.005392,-0.005392,0.006071,0.006071
8,0.0000,-0.005585,-0.005585,0.006071,0.006071
9,0.0002,-0.005406,-0.005406,0.006071,0.006071


The actual values and the predicted values are way too far from each other in both the datasets

We will now try a regressor  with a different activation function and check if it helps in any way

Activation Function: Leaky ReLu

In [53]:
from keras.layers.advanced_activations import LeakyReLU

# Trying with different number of nodes as well
regressor_leaky = Sequential()
regressor_leaky.add(Dense(75, kernel_initializer = 'normal',input_dim = 7))
regressor_leaky.add(LeakyReLU(alpha=0.05))
regressor_leaky.add(Dense(75, kernel_initializer= 'normal'))
regressor_leaky.add(LeakyReLU(alpha=0.05))
regressor_leaky.add(Dense(50, kernel_initializer= 'normal'))
regressor_leaky.add(LeakyReLU(alpha=0.05))
regressor_leaky.add(Dense(1, kernel_initializer = 'normal'))

# Compiling the regressor
regressor_leaky.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [54]:
# Fitting the ANN on training dataset with z-scores and then predicting the values for test dataset
regressor_leaky.fit(X_train_z, y_train_z, batch_size=5, epochs=100, verbose = 1)
y_pred_leaky_z = regressor_leaky.predict(X_test_z)

Epoch 1/100
24362/24362 [==============================] - 4s 184us/step - loss: 8.6120e-05 0s - loss: 9.3
Epoch 2/100
24362/24362 [==============================] - 4s 170us/step - loss: 8.5771e-05
Epoch 3/100
24362/24362 [==============================] - 4s 170us/step - loss: 8.5966e-05
Epoch 4/100
24362/24362 [==============================] - 4s 170us/step - loss: 8.4581e-05
Epoch 5/100
24362/24362 [==============================] - 4s 173us/step - loss: 8.5088e-05
Epoch 6/100
24362/24362 [==============================] - 4s 174us/step - loss: 8.2320e-05
Epoch 7/100
24362/24362 [==============================] - 4s 172us/step - loss: 7.8229e-05
Epoch 8/100
24362/24362 [==============================] - 4s 173us/step - loss: 8.2078e-05 
Epoch 9/100
24362/24362 [==============================] - 4s 178us/step - loss: 8.5297e-05 1s
Epoch 10/100
24362/24362 [==============================] - 5s 187us/step - loss: 7.9097e-05 0s - loss: 
Epoch 11/100
24362/24362 [======================

In [55]:
# Fitting the ANN on training dataset with just words and then predicting the values for test dataset
regressor_leaky.fit(X_train_w, y_train_w, batch_size=5, epochs=100, verbose = 1)
y_pred_leaky_w = regressor_leaky.predict(X_test_w)

Epoch 1/100
24362/24362 [==============================] - 5s 200us/step - loss: 8.7448e-05
Epoch 2/100
24362/24362 [==============================] - 5s 185us/step - loss: 8.8351e-05
Epoch 3/100
24362/24362 [==============================] - 5s 186us/step - loss: 8.6220e-05
Epoch 4/100
24362/24362 [==============================] - 5s 186us/step - loss: 8.6494e-05
Epoch 5/100
24362/24362 [==============================] - 5s 189us/step - loss: 8.6706e-05 0s
Epoch 6/100
24362/24362 [==============================] - 5s 189us/step - loss: 8.6958e-05
Epoch 7/100
24362/24362 [==============================] - 5s 188us/step - loss: 8.6519e-05
Epoch 8/100
24362/24362 [==============================] - 5s 194us/step - loss: 8.6435e-05
Epoch 9/100
24362/24362 [==============================] - 4s 181us/step - loss: 8.5375e-05
Epoch 10/100
24362/24362 [==============================] - 4s 181us/step - loss: 8.5992e-05 1s - - ETA: 0s - lo
Epoch 11/100
24362/24362 [==============================

In [57]:
y_pred_leaky_z=y_pred_leaky_z.reshape(6091,)
y_pred_leaky_w=y_pred_leaky_w.reshape(6091,)

temp={'Actual Values': y_test_z,'Leaky:Pred-Dataset with z-scores': y_pred_leaky_z,'Leaky:Pred-Dataset with words only': y_pred_w}
y_compare_leaky = pd.DataFrame(temp)

# Calculating the Mean Squared Error and adding the columns with the MSE
y_compare_leaky['MSE-> z-score'] = (((y_compare_leaky['Actual Values'].values - y_compare_leaky['Leaky:Pred-Dataset with z-scores'].values ) ** 2).mean() ** .5)
y_compare_leaky['MSE-> just words'] = (((y_compare_leaky['Actual Values'].values - y_compare_leaky['Leaky:Pred-Dataset with words only'].values ) ** 2).mean() ** .5)
y_compare_leaky.head(20)

,Actual Values,Leaky:Pred-Dataset with words only,Leaky:Pred-Dataset with z-scores,MSE-> z-score,MSE-> just words
0,0.0002,-0.005598,-0.000260,0.005217,0.006071
1,0.0000,-0.005475,-0.000215,0.005217,0.006071
2,0.0000,-0.005396,-0.000217,0.005217,0.006071
3,0.0000,-0.005301,0.000413,0.005217,0.006071
4,0.0000,-0.005349,-0.000015,0.005217,0.006071
5,0.0000,-0.004298,0.001075,0.005217,0.006071
6,0.0000,-0.005782,-0.000058,0.005217,0.006071
7,0.0000,-0.005392,-0.000100,0.005217,0.006071
8,0.0000,-0.005585,0.000130,0.005217,0.006071
9,0.0002,-0.005406,-0.000303,0.005217,0.006071


It is the same case with a different activation function and a different number of nodes.

Lower number of nodes has given us far superior results. 